In [2]:
from preprocessing import *
data_path = "data/game.csv"
X_train, X_test, y_train, y_test = prep_all(data_path)
# y_test.drop()

/Users/spencer/Desktop/Repos/nba-predictor/preprocessing.py:43: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['matchup', 'team_abbreviation', 'team_name', 'wl'], dtype='object')
  rolling_averages = grouped[columns_to_average].apply(lambda x: x.rolling(window=len(x), min_periods=1).mean().shift(1))
/Users/spencer/Desktop/Repos/nba-predictor/preprocessing.py:43: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  rolling_averages = grouped[columns_to_average].apply(lambda x: x.rolling

In [3]:
X_train = X_train.values.reshape(-1, 1, 135)  # Reshape to (32520, 1, 136)
X_test = X_test.values.reshape(-1, 1, 135)    # Reshape to (N, 1, 136), where N is the number of test samples


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna

In [6]:
train_data = TensorDataset(torch.tensor(X_train), torch.tensor(y_train.to_numpy()))
test_data = TensorDataset(torch.tensor(X_test), torch.tensor(y_test.to_numpy()))

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(test_data, batch_size=64, shuffle=False)


In [7]:

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        out = nn.functional.softmax(out, dim=1)
        return out


In [26]:
input_size = 135
output_size = 2
device = torch.device("cpu")
hidden_size = 32
num_layers = 4
learning_rate = 0.001
num_epochs = 20
model = RNN(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train(model, train_loader, optimizer, criterion, num_epochs=10):
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.float().to(device), labels.long().to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

train(model, train_loader, optimizer, criterion, num_epochs)


Epoch [1/20], Loss: 0.5743
Epoch [2/20], Loss: 0.5922
Epoch [3/20], Loss: 0.6731
Epoch [4/20], Loss: 0.7222
Epoch [5/20], Loss: 0.5935
Epoch [6/20], Loss: 0.5728
Epoch [7/20], Loss: 0.5434
Epoch [8/20], Loss: 0.6114
Epoch [9/20], Loss: 0.5644
Epoch [10/20], Loss: 0.6399
Epoch [11/20], Loss: 0.6092
Epoch [12/20], Loss: 0.4779
Epoch [13/20], Loss: 0.6133
Epoch [14/20], Loss: 0.6097
Epoch [15/20], Loss: 0.5379
Epoch [16/20], Loss: 0.5967
Epoch [17/20], Loss: 0.5202
Epoch [18/20], Loss: 0.6044
Epoch [19/20], Loss: 0.5976
Epoch [20/20], Loss: 0.5545


In [27]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in valid_loader:
        inputs, labels = inputs.float(), labels.long()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    val_acc = correct / total
    print(f'Test Accuracy: {100 * correct / total:.2f}%')


Test Accuracy: 65.51%


In [20]:
def train_model(model, train_loader, criterion, optimizer, device):
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.float().to(device), labels.long().to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        

def evaluate_model(model, valid_loader, device):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in valid_loader:
            inputs, labels = inputs.float(), labels.long()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        val_acc = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Acc: {100 * correct / total:.2f}%')

def objective(trial):
    # Define hyperparameters
    hidden_size = trial.suggest_int("hidden_size", 16, 512)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2)

    # Create the RNN model
    model = RNN(input_size, hidden_size, output_size, num_layers).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    num_epochs = 20
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.float().to(device), labels.long().to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for inputs, labels in valid_loader:
                inputs, labels = inputs.float(), labels.long()
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            val_acc = correct / total * 100
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Validation Acc: {100 * correct / total:.2f}%')

        trial.report(val_acc, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return val_acc


In [21]:
# Specify the number of trials
num_trials = 100
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=num_trials)

# Print the best trial
best_trial = study.best_trial
print(f"Best trial: {best_trial.number}, accuracy: {best_trial.value}")
print(f"Best hyperparameters: {best_trial.params}")


[I 2023-05-03 03:15:21,850] A new study created in memory with name: no-name-a6182ade-ed06-49ef-92a8-46a1f8aea1c9


Epoch [1/20], Loss: 0.5440, Validation Acc: 66.28%
Epoch [2/20], Loss: 0.6488, Validation Acc: 66.65%
Epoch [3/20], Loss: 0.5920, Validation Acc: 66.45%
Epoch [4/20], Loss: 0.6001, Validation Acc: 66.87%
Epoch [5/20], Loss: 0.5309, Validation Acc: 66.38%
Epoch [6/20], Loss: 0.5409, Validation Acc: 66.61%
Epoch [7/20], Loss: 0.5811, Validation Acc: 66.80%
Epoch [8/20], Loss: 0.6706, Validation Acc: 66.77%
Epoch [9/20], Loss: 0.5863, Validation Acc: 66.22%
Epoch [10/20], Loss: 0.6089, Validation Acc: 66.75%
Epoch [11/20], Loss: 0.7111, Validation Acc: 65.10%
Epoch [12/20], Loss: 0.6184, Validation Acc: 66.26%
Epoch [13/20], Loss: 0.6020, Validation Acc: 66.56%
Epoch [14/20], Loss: 0.6398, Validation Acc: 65.56%
Epoch [15/20], Loss: 0.5625, Validation Acc: 66.10%
Epoch [16/20], Loss: 0.6017, Validation Acc: 65.73%
Epoch [17/20], Loss: 0.6390, Validation Acc: 66.37%
Epoch [18/20], Loss: 0.4970, Validation Acc: 65.55%
Epoch [19/20], Loss: 0.7054, Validation Acc: 65.96%


[I 2023-05-03 03:22:14,821] Trial 0 finished with value: 65.7723983029698 and parameters: {'hidden_size': 310, 'num_layers': 3, 'learning_rate': 0.001050439342929995}. Best is trial 0 with value: 65.7723983029698.


Epoch [20/20], Loss: 0.5095, Validation Acc: 65.77%
Epoch [1/20], Loss: 0.6108, Validation Acc: 65.92%
Epoch [2/20], Loss: 0.5598, Validation Acc: 65.92%
Epoch [3/20], Loss: 0.6975, Validation Acc: 66.96%
Epoch [4/20], Loss: 0.5277, Validation Acc: 66.62%
Epoch [5/20], Loss: 0.5475, Validation Acc: 65.01%
Epoch [6/20], Loss: 0.5140, Validation Acc: 65.86%
Epoch [7/20], Loss: 0.6293, Validation Acc: 65.71%
Epoch [8/20], Loss: 0.5466, Validation Acc: 66.75%
Epoch [9/20], Loss: 0.6089, Validation Acc: 65.80%
Epoch [10/20], Loss: 0.5591, Validation Acc: 66.28%
Epoch [11/20], Loss: 0.6337, Validation Acc: 66.32%
Epoch [12/20], Loss: 0.6436, Validation Acc: 66.77%
Epoch [13/20], Loss: 0.6143, Validation Acc: 66.07%
Epoch [14/20], Loss: 0.6855, Validation Acc: 66.37%
Epoch [15/20], Loss: 0.7107, Validation Acc: 65.49%
Epoch [16/20], Loss: 0.5604, Validation Acc: 66.27%
Epoch [17/20], Loss: 0.5689, Validation Acc: 65.83%
Epoch [18/20], Loss: 0.5809, Validation Acc: 65.35%
Epoch [19/20], Loss: 

[I 2023-05-03 03:28:28,704] Trial 1 finished with value: 66.4711754429748 and parameters: {'hidden_size': 270, 'num_layers': 3, 'learning_rate': 0.001569712781075145}. Best is trial 1 with value: 66.4711754429748.


...

Epoch [20/20], Loss: 0.6028, Validation Acc: 66.60%


[I 2023-05-03 09:58:13,466] Trial 98 pruned. 


Epoch [1/20], Loss: 0.6352, Validation Acc: 66.63%


[I 2023-05-03 09:58:18,055] Trial 99 pruned. 


Epoch [1/20], Loss: 0.6595, Validation Acc: 63.71%
Best trial: 49, accuracy: 67.28225605190916
Best hyperparameters: {'hidden_size': 72, 'num_layers': 1, 'learning_rate': 0.0006180540563092695}


In [24]:
input_size = 135
output_size = 2
device = torch.device("cpu")
hidden_size = 72
num_layers = 1
learning_rate = 0.0006180540563092695
num_epochs = 20
model = RNN(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train(model, train_loader, optimizer, criterion, num_epochs=10):
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.float().to(device), labels.long().to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

train(model, train_loader, optimizer, criterion, num_epochs)


Epoch [1/20], Loss: 0.6233
Epoch [2/20], Loss: 0.5727
Epoch [3/20], Loss: 0.6133
Epoch [4/20], Loss: 0.5486
Epoch [5/20], Loss: 0.6518
Epoch [6/20], Loss: 0.5546
Epoch [7/20], Loss: 0.5295
Epoch [8/20], Loss: 0.6521
Epoch [9/20], Loss: 0.6389
Epoch [10/20], Loss: 0.5494
Epoch [11/20], Loss: 0.6185
Epoch [12/20], Loss: 0.5393
Epoch [13/20], Loss: 0.5371
Epoch [14/20], Loss: 0.5875
Epoch [15/20], Loss: 0.6334
Epoch [16/20], Loss: 0.5529
Epoch [17/20], Loss: 0.6070
Epoch [18/20], Loss: 0.6308
Epoch [19/20], Loss: 0.5473
Epoch [20/20], Loss: 0.6353


In [25]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in valid_loader:
        inputs, labels = inputs.float(), labels.long()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    val_acc = correct / total
    print(f'Test Accuracy: {100 * correct / total:.2f}%')


Test Accuracy: 67.01%
